In [1]:
import BioSimSpace as bss
import glob
import BioSimSpace._Exceptions

INFO:rdkit:Enabling RDKit 2022.09.1 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
somd_path = "/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_"
repeats = ["1", "2", "3"]

In [3]:
analysis_path = "/home/jguven/projects/metalloenzymes/starmap/afe/"

In [4]:
unbound_repeats, bound_repeats = [], []
for repeat in repeats:
    somd_folder = somd_path + repeat + "/"
    analysis_file = analysis_path + "somd_repeat_analysis_" + repeat + ".txt"
    paths = sorted(glob.glob(somd_folder + "*/"))
    unbound_pmfs, bound_pmfs = [], []
    for path in paths:
        unbound_pmfs.append(path)
    unbound_repeats.append(unbound_pmfs)

In [5]:
import numpy as np
np.shape(unbound_repeats)

(3, 15)

In [6]:
unbound_repeats, bound_repeats = [], []
for repeat in repeats:
    somd_folder = somd_path + repeat + "/"
    analysis_file = analysis_path + "somd_repeat_analysis_" + repeat + ".txt"
    paths = sorted(glob.glob(somd_folder + "*/"))
    unbound_pmfs, bound_pmfs = [], []
    for path in paths:
        ligand = path.split("/")[-2]
        unbound_directory = somd_folder + ligand + "/unbound/"
        bound_directory = somd_folder + ligand + "/bound/"     
        transformation = path.split("/")[-2]
        try:
            pmf_unbound, overlap_matrix_unbound = bss.FreeEnergy.Relative.analyse(unbound_directory)
            # unbound_pmfs.append(pmf_unbound)
            pmf_bound, overlap_matrix_bound = bss.FreeEnergy.Relative.analyse(bound_directory)
            # bound_pmfs.append(pmf_bound)
            # print(pmf_unbound)
            print(repeat)
            print(path)
        except BioSimSpace._Exceptions.AnalysisError as e:
            print(e)   
    # unbound_repeats.append(unbound_pmfs)
    # bound_repeats.append(bound_pmfs)

1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_10/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_11/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_12/
SOMD free-energy analysis failed!
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_14/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_15/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_16/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_2/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_3/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_4/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_5/
1
/home/jguven/projects/metalloenzymes/starmap/outputs/with_min/SOMD_1/lig_1~lig_6/
1
/home/jguven/projects/metalloenzym

In [7]:
np.shape(unbound_repeats)

(0,)

In [8]:
# unbound_pmfs, bound_pmfs = [], []

# for repeat in repeats:

#     somd_folder = somd_path + repeat + "/"
#     analysis_file = analysis_path + "somd_repeat_analysis_" + repeat + ".txt"
#     paths = sorted(glob.glob(somd_folder + "*/"))
    
#     unbound_pmfs, bound_pmfs = [], []
    
#     for path in paths:
#         ligand = path.split("/")[-2]
#         unbound_directory = somd_folder + ligand + "/unbound/"
#         bound_directory = somd_folder + ligand + "/bound/"     
#         transformation = path.split("/")[-2]
#         try:
#             pmf_unbound, overlap_matrix_unbound = bss.FreeEnergy.Relative.analyse(unbound_directory)
#             pmf_bound, overlap_matrix_bound = bss.FreeEnergy.Relative.analyse(bound_directory)
            
#         except BioSimSpace._Exceptions.AnalysisError as e:
#             print(e)   


In [9]:
with open(analysis_file, "w") as file:            
    try:  
        free_energy_difference, free_energy_error = bss.FreeEnergy.Relative.difference(pmf_bound, pmf_unbound)
        file.write(transformation + "\n")
        file.write(f"{free_energy_difference} \u00B1 {free_energy_error} \n")   
    except BioSimSpace._Exceptions.AnalysisError as e:
        print(e)
    except IndexError as e:
        print(str(e)+"\n")
    except ValueError as e:
        print(str(e)+"\n")